# 2. 擴展鏈 (Chains Extended) - Ollama 版本

本範例展示如何在鏈中加入自定義處理步驟，建立一個智能郵件回覆系統，包含格式化和品質檢查。

## 學習重點
- 使用 RunnableLambda 添加自定義邏輯
- 學習如何擴展基本的鏈功能
- 理解鏈的模組化特性
- 掌握多步驟處理的技巧


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_ollama.llms import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型（統一使用 llama3.2:latest）
model = OllamaLLM(model="llama3.2:latest")

In [7]:
# 定義提示模板 - 建立郵件回覆系統
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調回覆。"),
        ("human", "請回覆這封客戶郵件：\n\n{email_content}"),
    ]
)


In [8]:
# 定義自定義處理步驟
# 使用 RunnableLambda 來添加自定義邏輯

# 步驟 1: 格式化郵件回覆（添加標題和簽名）
def format_email_reply(reply):
    """格式化郵件回覆，添加專業的郵件格式"""
    formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：{__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M')}
"""
    return formatted_reply.strip()

format_email = RunnableLambda(format_email_reply)

# 步驟 2: 添加品質檢查和統計資訊
def add_quality_check(formatted_reply):
    """添加品質檢查和統計資訊"""
    word_count = len(formatted_reply.split())
    char_count = len(formatted_reply)
    
    quality_info = f"""
📊 回覆統計：
- 字數：{word_count} 字
- 字元數：{char_count} 字元
- 回覆品質：{'✅ 良好' if word_count >= 50 else '⚠️ 建議增加內容'}

{formatted_reply}
"""
    return quality_info

quality_check = RunnableLambda(add_quality_check)

print("自定義處理步驟已定義：")
print("1. format_email: 格式化郵件回覆")
print("2. quality_check: 添加品質檢查和統計資訊")


自定義處理步驟已定義：
1. format_email: 格式化郵件回覆
2. quality_check: 添加品質檢查和統計資訊


In [9]:
# 建立擴展的鏈
# 鏈的流程：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查
chain = prompt_template | model | StrOutputParser() | format_email | quality_check

print("擴展鏈已建立完成！")
print("鏈的結構：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查")


擴展鏈已建立完成！
鏈的結構：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查


In [10]:
# 執行擴展鏈 - 測試郵件回覆系統
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。

王小明
"""

result = chain.invoke({"email_content": customer_email})

# 輸出結果
print("=" * 60)
print("智能郵件回覆系統處理結果：")
print("=" * 60)
print(result)


智能郵件回覆系統處理結果：

📊 回覆統計：
- 字數：69 字
- 字元數：878 字元
- 回覆品質：✅ 良好

親愛的客戶，

尊敬的王先生／女士：

您好！  
感謝您選擇並信任我們的產品，並抽空與我們聯繫。  
首先，對於您收到的包裝損壞以及產品功能與說明書不符的情況，我們深表歉意，並非常重視您的使用體驗。

為協助您盡快解決此問題，以下提供兩種處理方式，您可依照方便與需求自行選擇：

| 方案 | 具體步驟 | 需要準備的材料 |
|------|----------|---------------|
| **退貨** | 1. 於本郵件回覆中告知「退貨」並附上訂單號。<br>2. 我們將在 2 個工作日內寄出退貨標籤與說明。<br>3. 將產品、附件及原包裝完整包裝，並貼上退貨標籤寄回。<br>4. 收到商品後，我們將於 3 個工作日內完成退款（或退回已付費款項）。 | 商品原包裝、產品、說明書、購買收據或訂單號 |
| **換貨** | 1. 於本郵件回覆中告知「換貨」並附上訂單號。<br>2. 我們將協助您寄回有瑕疵的商品，並於確認無誤後寄送全新同型號產品。<br>3. 如需加急配送，我們亦可提供相應方案。 | 同上（退貨） |

**備註：**  
- 為確保處理速度，我們建議您先拍攝包裝損壞部位及產品功能不符的截圖，並一併發送至此郵件。  
- 若您更願意直接致電客服，我們的專線為 **(02) 1234‑5678**，週一至週五 9:00–17:00（含午休）皆可。  

再次為此次不愉快經歷向您致歉，我們將以最快速度為您處理。若您有任何進一步的疑問或需求，隨時歡迎回覆此郵件或撥打客服專線。

期待能為您盡快解決問題，謝謝您對我們的支持與理解！

祝 您安好、生活愉快！

此致  
敬禮

**客戶服務團隊**  
ABC 企業有限公司  
客服專線： (02) 1234‑5678  
客服信箱： support@abc.com  

---  

*本郵件由自動系統轉發，若您已處理完畢，請忽略此訊息。*

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：2025-10-01 15:11



## 💡 重點說明

### 擴展鏈的優勢

1. **模組化設計**: 每個處理步驟都是獨立的，可以輕鬆添加、移除或修改
2. **可重用性**: 自定義的處理步驟可以在不同的鏈中重複使用
3. **靈活性**: 可以根據需求組合不同的處理步驟

### RunnableLambda 的使用

```python
# 基本語法
custom_step = RunnableLambda(lambda x: 自定義處理函數(x))

# 實際範例
format_email = RunnableLambda(format_email_reply)
quality_check = RunnableLambda(add_quality_check)
```

### 鏈的組合方式

```python
# 使用管道符號串聯多個步驟
chain = (
    prompt_template 
    | model 
    | StrOutputParser() 
    | format_email 
    | quality_check
)
```

## 🔧 實際應用場景

- **客服系統**: 自動回覆客戶郵件
- **內容管理**: 格式化生成內容
- **品質控制**: 自動檢查內容品質
- **報告生成**: 添加統計資訊和格式
